In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import os
import re
from pathlib import Path
from glob import glob
from loguru import logger
import cdblib
import numpy as np

In [3]:
dir_path = os.getcwd()
if not os.path.exists(f'{dir_path}/output'):
    result_path = os.makedirs(f'{dir_path}/output')


In [4]:
dir_path = os.getcwd()
folders = sorted(Path(dir_path).glob('D/C5002 LEFT/'))
# folders = folders[1:3]
folders

[PosixPath('/home/pinky/Documents/p_projects/rabina_file/D/C5002 LEFT')]

In [5]:
for i in folders:
    files = sorted(Path(i).glob('*.TXT'))
    folder_name = str(i).split('/')[-1]
    logger.info(f"Calculating values for folder: {folder_name} ...")
    results = []
    for file in files:
        file_name = os.path.splitext(file.name)[0]
        angle = file_name.split('_')[-1]
        logger.info(f"Reading for file: {file_name} and angle: {angle}")
        with open(file, 'r') as f:
            lines = f.readlines()
            del lines[:2]
            final_lines = [re.split('\s+', l.strip()) for l in lines]
            df = pd.DataFrame(final_lines[1:], columns=final_lines[0])
#             df = df.drop(columns=['NODE'])
            values = [
                    angle,
                    int(
                        float(
                            df[
                                df["EQUIVALENTSTRESS"].astype("float64")
                                == df["EQUIVALENTSTRESS"].astype("float64").max()
                            ]["NODE"].iloc[0]
                        )
                    ),
                    df["EQUIVALENTSTRESS"].astype("float64").max(),
                    int(
                        float(
                            df[
                                df["EQUIVALENTSTRESS"].astype("float64")
                                == df["EQUIVALENTSTRESS"].astype("float64").min()
                            ]["NODE"].iloc[0]
                        )
                    ),
                    df["EQUIVALENTSTRESS"].astype("float64").min(),
                    int(
                        float(
                            df[
                                df["PRINCIPALSTRESS"].astype("float64")
                                == df["PRINCIPALSTRESS"].astype("float64").max()
                            ]["NODE"].iloc[0]
                        )
                    ),
                    df["PRINCIPALSTRESS"].astype("float64").max(),
                    int(
                        float(
                            df[
                                df["PRINCIPALSTRESS"].astype("float64")
                                == df["PRINCIPALSTRESS"].astype("float64").min()
                            ]["NODE"].iloc[0]
                        )
                    ),
                    df["PRINCIPALSTRESS"].astype("float64").min(),
                    int(
                        float(
                            df[
                                df["EQVSTRAIN"].astype("float64")
                                == df["EQVSTRAIN"].astype("float64").max()
                            ]["NODE"].iloc[0]
                        )
                    ),
                    df["EQVSTRAIN"].astype("float64").max(),
                    int(
                        float(
                            df[
                                df["EQVSTRAIN"].astype("float64")
                                == df["EQVSTRAIN"].astype("float64").min()
                            ]["NODE"].iloc[0]
                        )
                    ),
                    df["EQVSTRAIN"].astype("float64").min(),
                    int(
                        float(
                            df[
                                df["PRINCIPALSTRAIN"].astype("float64")
                                == df["PRINCIPALSTRAIN"].astype("float64").max()
                            ]["NODE"].iloc[0]
                        )
                    ),
                    df["PRINCIPALSTRAIN"].astype("float64").max(),
                    int(
                        float(
                            df[
                                df["PRINCIPALSTRAIN"].astype("float64")
                                == df["PRINCIPALSTRAIN"].astype("float64").min()
                            ]["NODE"].iloc[0]
                        )
                    ),
                    df["PRINCIPALSTRAIN"].astype("float64").min(),
                ]
            logger.info(f"Values for angle: {angle} calculated!")
            results.append(values)
    #         sorting results on the basis of angle
            sorted_result = sorted(results, key=lambda x: int(x[0]))
    results_df = pd.DataFrame(sorted_result, columns=['Angle', 
                                                      "NODE_EQUIVALENTSTRESS_max", "EQUIVALENTSTRESS_max", 
                                                      "NODE_EQUIVALENTSTRESS_min", "EQUIVALENTSTRESS_min", 
                                                      "NODE_PRINCIPALSTRESS_max", "PRINCIPALSTRESS_max", 
                                                      "NODE_PRINCIPALSTRESS_min", "PRINCIPALSTRESS_min", 
                                                      "NODE_EQVSTRAIN_max", "EQVSTRAIN_max", 
                                                      "NODE_EQVSTRAIN_min", "EQVSTRAIN_min", 
                                                      "NODE_PRINCIPALSTRAIN_max", "PRINCIPALSTRAIN_max", 
                                                      "NODE_PRINCIPALSTRAIN_min", "PRINCIPALSTRAIN_min",])
    if not os.path.exists(f'{dir_path}/output'):
        os.makedirs(f'{dir_path}/output')
#     results_df.to_csv(f"{dir_path}/output/{folder_name}.csv")

2020-06-24 08:02:50.961 | INFO     | __main__:<module>:4 - Calculating values for folder: C5002 LEFT ...
2020-06-24 08:02:50.963 | INFO     | __main__:<module>:9 - Reading for file: TRY_0 and angle: 0
2020-06-24 08:02:51.298 | INFO     | __main__:<module>:91 - Values for angle: 0 calculated!
2020-06-24 08:02:51.299 | INFO     | __main__:<module>:9 - Reading for file: TRY_10 and angle: 10
2020-06-24 08:02:51.407 | INFO     | __main__:<module>:91 - Values for angle: 10 calculated!
2020-06-24 08:02:51.407 | INFO     | __main__:<module>:9 - Reading for file: TRY_120 and angle: 120
2020-06-24 08:02:51.508 | INFO     | __main__:<module>:91 - Values for angle: 120 calculated!
2020-06-24 08:02:51.509 | INFO     | __main__:<module>:9 - Reading for file: TRY_15 and angle: 15
2020-06-24 08:02:51.652 | INFO     | __main__:<module>:91 - Values for angle: 15 calculated!
2020-06-24 08:02:51.653 | INFO     | __main__:<module>:9 - Reading for file: TRY_20 and angle: 20
2020-06-24 08:02:51.753 | INFO   

# find material and element from cdb file

In [6]:
for i in folders:
    cdb_file = sorted(Path(i).glob('*.cdb'))
    with open(cdb_file[0], 'r') as f:
        data = f.readlines()

In [7]:
for i, text in enumerate(data):
    if "Elements" in text:
        start_element = i
    if "Component" in text:
        start_component = i
    if "MATERIALS" in text:
        start_material = i
    if "Nodes for the whole assembly" in text:
        start_nodes_xyz = i
nodes_xyz_data = [text.strip() for i, text in enumerate(data) if (i >= start_nodes_xyz and i < start_element)]
element_data = [text.strip() for i, text in enumerate(data) if (i >= start_element and i < start_component)]
material_data = [text.strip() for i, text in enumerate(data) if (i >= start_material)]

In [8]:
nodes_xyz_data = [re.split('\s+', l.strip()) for l in nodes_xyz_data[3:-2]]
nodes_xyz_df = pd.DataFrame(nodes_xyz_data, columns=['NODE_PRINCIPALSTRAIN_max', 'A', 'B', 'x', 'y', 'z'])
nodes_xyz_df['NODE_PRINCIPALSTRAIN_max']=nodes_xyz_df['NODE_PRINCIPALSTRAIN_max'].astype(int)
nodes_xyz_df['x'] = nodes_xyz_df['x'].astype(float)
nodes_xyz_df['y'] = nodes_xyz_df['y'].astype(float)
nodes_xyz_df['z'] = nodes_xyz_df['z'].astype(float)

In [9]:
principal_strain_df = results_df[["NODE_PRINCIPALSTRAIN_max", "PRINCIPALSTRAIN_max"]]

# extracting x,y,z from node data to principal strain

In [10]:
p_strain_xyz = pd.merge(principal_strain_df, nodes_xyz_df, on='NODE_PRINCIPALSTRAIN_max')

In [11]:
p_strain_xyz['x_max'] = p_strain_xyz['x'].astype(float) + 2
p_strain_xyz['x_min'] = p_strain_xyz['x'].astype(float) - 2
p_strain_xyz['y_max'] = p_strain_xyz['y'].astype(float) + 4
p_strain_xyz['y_min'] = p_strain_xyz['y'].astype(float) - 4
p_strain_xyz['z_max'] = p_strain_xyz['z'].astype(float) + 2
p_strain_xyz['z_min'] = p_strain_xyz['z'].astype(float) - 2

# calculating extra nodes

In [12]:
extra_principal_strain_node = [[]]*len(results_df)
node_df_node = nodes_xyz_df['NODE_PRINCIPALSTRAIN_max'].values # all nodes values
node_df_x = nodes_xyz_df['x'].values
node_df_y = nodes_xyz_df['y'].values
node_df_z = nodes_xyz_df['z'].values
x_max = p_strain_xyz['x_max'].values
x_min = p_strain_xyz['x_min'].values
y_max = p_strain_xyz['y_max'].values
y_min = p_strain_xyz['y_min'].values
z_max = p_strain_xyz['z_max'].values
z_min = p_strain_xyz['z_min'].values

for i in range(len(x_max)):
    for j in np.where(((x_min[i] <= node_df_x) & (node_df_x <= x_max[i])) & 
                      ((y_min[i] <= node_df_y) & (node_df_y <= y_max[i])) & 
                      ((z_min[i] <= node_df_z) & (node_df_z <= z_max[i]))): 
        extra_principal_strain_node[i] = node_df_node[j]
results_df['EXTRA_NODE_PRINCIPALSTRAIN'] = extra_principal_strain_node

# processing elements

In [13]:
element_data = [re.split('\s+', l.strip()) for l in element_data[4:-1]]

In [14]:
element_df = pd.DataFrame(element_data, columns=['Material_value', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'Node4', 'Node3', 'Node2', 'Node1'])

In [15]:
material_val = (element_df['Material_value'].astype(float).astype(int)).values
nodes1 = (element_df['Node1'].astype(int)).values
nodes2 = (element_df['Node2'].astype(int)).values
all_nodes_p_strain = [i.tolist() for i in results_df['EXTRA_NODE_PRINCIPALSTRAIN'].tolist()]

In [16]:
nodes_p_strain_test = results_df['NODE_PRINCIPALSTRAIN_max'].values
for i, val in enumerate(nodes_p_strain_test):
    all_nodes_p_strain[i].append(val)

In [17]:
all_nodes_p_strain= [set(row) for row in all_nodes_p_strain]

In [18]:
results_df["EXTRA_NODE_PRINCIPALSTRAIN"] = all_nodes_p_strain

In [19]:
node1_material_list = dict(zip(nodes1, material_val))
node2_material_list = dict(zip(nodes2, material_val))

In [20]:
material_values_list = []
for row in all_nodes_p_strain:
    temp_list = []
    for item in row:
        try:
            temp_list.append(node1_material_list[item])
        except KeyError:
            temp_list.append(node2_material_list[item])
    material_values_list.append(set(temp_list))
            

In [21]:
results_df['MATERIAL_PRINCIPALSTRAIN'] = material_values_list

# calculate final value from material

In [22]:
material_data_f = [re.split(',', l.strip()) for l in material_data[1:-8]]
material_data_new = [i for i in material_data_f if "EX" in i]

In [23]:
material_df = pd.DataFrame(material_data_new, columns=['A', 'B', 'C', 'D', 'Material_value', 'F', 'Final_value', 'H'])

In [24]:
material_value = material_df['Material_value'].astype(int).values
final_value = material_df['Final_value'].astype(float).values

In [25]:
material_final_val_list = dict(zip(material_value, final_value))

In [26]:
material_p_strain = results_df['MATERIAL_PRINCIPALSTRAIN'].values

In [27]:
final_value_list = []
for row in material_p_strain:
    temp_list = []
    for items in row:
        temp_list.append(material_final_val_list[items])
    final_value_list.append(set(temp_list))

In [28]:
results_df['FINAL_VALUE_PRINCIPAL_STRAIN'] = final_value_list

In [31]:
# subset_df = results_df[["Angle", "EXTRA_NODE_PRINCIPALSTRAIN"]]

In [41]:
# s_angle = subset_df['Angle'].astype(int).values
# s_extra_node = subset_df['EXTRA_NODE_PRINCIPALSTRAIN'].values

In [83]:
files = sorted(Path(folders[0]).glob('*.TXT'))
angles = [int(os.path.splitext(file.name)[0].split('_')[-1]) for file in files]
files_dict = dict(zip(angles, files))
p_strain_values = []
for angle, filepath in files_dict.items():
    nodes = results_df[results_df["Angle"].astype(int)==angle]["EXTRA_NODE_PRINCIPALSTRAIN"].values
    with open(filepath, 'r') as f:
        lines = f.readlines()
        del lines[:2]
        final_lines = [re.split('\s+', l.strip()) for l in lines]
        df = pd.DataFrame(final_lines[1:], columns=final_lines[0])
        p_strain_extra_nodes = [float(df[df['NODE'].astype(float).astype(int)==node]['PRINCIPALSTRAIN'].iloc[0]) for node in nodes[0]]
        p_strain_values.append((angle, p_strain_extra_nodes))
sorted_p_values = sorted(p_strain_values, key=lambda x: int(x[0]))
sorted_strain_values = [strain_val for angle, strain_val in sorted_p_values]
results_df['EXTRA_NODE_PRINCIPAL_STRAIN_VALUE'] = sorted_strain_values

In [84]:
results_df

,Angle,NODE_EQUIVALENTSTRESS_max,EQUIVALENTSTRESS_max,NODE_EQUIVALENTSTRESS_min,EQUIVALENTSTRESS_min,NODE_PRINCIPALSTRESS_max,PRINCIPALSTRESS_max,NODE_PRINCIPALSTRESS_min,PRINCIPALSTRESS_min,NODE_EQVSTRAIN_max,...,NODE_EQVSTRAIN_min,EQVSTRAIN_min,NODE_PRINCIPALSTRAIN_max,PRINCIPALSTRAIN_max,NODE_PRINCIPALSTRAIN_min,PRINCIPALSTRAIN_min,EXTRA_NODE_PRINCIPALSTRAIN,MATERIAL_PRINCIPALSTRAIN,FINAL_VALUE_PRINCIPAL_STRAIN,EXTRA_NODE_PRINCIPAL_STRAIN_VALUE
0,0,4970,22.529,68,0.0,4879,16.058,5690,-5.9231,9351,...,68,0.0,11077,0.004269,9422,-1.230800e-06,"{11077, 11532, 7052, 9847, 12185}","{8, 17, 10, 12}","{1961.16, 756.31, 445.43, 223.46}","[0.0042692, 0.0030323, 0.0036156, 0.0037267, 0..."
1,10,4970,20.395,68,0.0,10572,12.868,12099,-4.8742,9351,...,68,0.0,11077,0.003645,5424,-7.985800e-07,"{11077, 11532, 7052, 9847, 12185}","{8, 17, 10, 12}","{1961.16, 756.31, 445.43, 223.46}","[0.0036448, 0.0025624, 0.0031072, 0.0031618, 0..."
2,15,3299,126.670,68,0.0,10986,138.690,7830,-21.4190,6601,...,68,0.0,6601,0.027234,3785,-1.170500e-05,"{6601, 10444}","{9, 11}","{323.76, 589.34}","[0.027234, 0.025976]"
3,20,10572,36.491,68,0.0,10572,43.400,10201,-7.3483,9351,...,68,0.0,7124,0.005401,5424,-1.564000e-05,"{6601, 7124, 10444}","{9, 11, 4}","{13.86, 323.76, 589.34}","[0.0026576, 0.0054006, 0.002621]"
4,30,9841,45.812,68,0.0,9913,13.054,9585,-42.9520,9351,...,68,0.0,8916,0.007165,3629,-9.147800e-07,"{8916, 7124}","{4, 5}","{40.18, 13.86}","[0.0071652, 0.0011206]"
5,45,3299,341.920,68,0.0,11339,371.260,7830,-66.7640,6601,...,68,0.0,6601,0.070610,3785,-7.029700e-05,"{10848, 11969, 3651, 6601, 8916, 9812}","{5, 6, 8, 9, 12, 14}","{323.76, 40.18, 1162.31, 82.88, 756.31, 223.46}","[0.006932, 0.0042765, 0.014422, 0.07061, 0.010..."
6,60,11073,30.352,68,0.0,11143,11.870,11032,-29.9790,9351,...,68,0.0,10113,0.007335,10199,-5.175400e-04,"{10848, 10113, 10114, 10115, 11969, 10279, 770...","{6, 11, 12, 13, 17, 22, 24}","{1961.16, 589.34, 82.88, 947.07, 756.31, 4787....","[0.0034973, 0.0073354, 0.0036055, 0.0019233, 0..."
7,90,11073,32.984,68,0.0,12572,33.182,11073,-26.0160,9351,...,68,0.0,10113,0.009301,68,0.000000e+00,"{10848, 10113, 10114, 10115, 11969, 10279, 770...","{6, 11, 12, 13, 17, 22, 24}","{1961.16, 589.34, 82.88, 947.07, 756.31, 4787....","[0.0035072, 0.0093014, 0.0040904, 0.0023109, 0..."
8,120,4970,56.338,68,0.0,12572,63.541,11073,-22.6890,12003,...,68,0.0,6869,0.009714,11858,-2.664000e-06,"{12220, 9332, 6869, 9367, 4348, 4350}","{9, 11, 12, 13, 15}","{323.76, 589.34, 947.07, 756.31, 1402.67}","[0.0073865, 0.0092156, 0.009714, 0.0091397, 0...."


In [85]:
results_df.to_csv(f'{dir_path}/output/{folder_name}.csv')

# for finding out all the principal_strains of extra nodes

In [86]:
node_lists = results_df["EXTRA_NODE_PRINCIPALSTRAIN"].values

In [92]:
unique_extra_nodes = set([n for node in node_lists for n in node])

In [83]:
def get_p_strain(node):
    p_strain_values = []
    for i in folders:
        files = sorted(Path(i).glob('*.TXT'))
        folder_name = str(i).split('/')[-1]
        logger.info(f"Calculating values for folder: {folder_name} ...")
        results = []
        for file in files:
            file_name = os.path.splitext(file.name)[0]
            angle = file_name.split('_')[-1]
            logger.info(f"Reading for file: {file_name} and angle: {angle}")
            with open(file, 'r') as f:
                lines = f.readlines()
                del lines[:2]
                final_lines = [re.split('\s+', l.strip()) for l in lines]
                df = pd.DataFrame(final_lines[1:], columns=final_lines[0])
                extra_node_strain = float(df[df['NODE'].astype(float).astype(int)==node]['PRINCIPALSTRAIN'].iloc[0])
                p_strain_values.append((angle, extra_node_strain))
    sorted_p_values = sorted(p_strain_values, key=lambda x: int(x[0]))
    return sorted_p_values

In [97]:
all_nodes_p_values = []
for node in unique_extra_nodes:
    p_strain_values = get_p_strain(node)
    all_nodes_p_values.append((node, p_strain_values))

2020-06-23 08:43:23.271 | INFO     | __main__:get_p_strain:6 - Calculating values for folder: C5002 LEFT ...
2020-06-23 08:43:23.272 | INFO     | __main__:get_p_strain:11 - Reading for file: TRY_0 and angle: 0
2020-06-23 08:43:23.319 | INFO     | __main__:get_p_strain:11 - Reading for file: TRY_10 and angle: 10
2020-06-23 08:43:23.368 | INFO     | __main__:get_p_strain:11 - Reading for file: TRY_120 and angle: 120
2020-06-23 08:43:23.439 | INFO     | __main__:get_p_strain:11 - Reading for file: TRY_15 and angle: 15
2020-06-23 08:43:23.499 | INFO     | __main__:get_p_strain:11 - Reading for file: TRY_20 and angle: 20
2020-06-23 08:43:23.558 | INFO     | __main__:get_p_strain:11 - Reading for file: TRY_30 and angle: 30
2020-06-23 08:43:23.751 | INFO     | __main__:get_p_strain:11 - Reading for file: TRY_45 and angle: 45
2020-06-23 08:43:23.812 | INFO     | __main__:get_p_strain:11 - Reading for file: TRY_60 and angle: 60
2020-06-23 08:43:23.866 | INFO     | __main__:get_p_strain:11 - Rea

In [100]:
p_strain_df = pd.DataFrame(all_nodes_p_values, columns=['NODE', 'ANGLE_PRINCIPAL_STRAIN'])

In [102]:
p_strain_df.to_csv(f'{dir_path}/output/{folder_name}_pstrain.csv')

In [84]:
get_p_strain(12185)

2020-06-23 08:33:48.221 | INFO     | __main__:get_p_strain:6 - Calculating values for folder: C5002 LEFT ...
2020-06-23 08:33:48.223 | INFO     | __main__:get_p_strain:11 - Reading for file: TRY_0 and angle: 0
2020-06-23 08:33:48.297 | INFO     | __main__:get_p_strain:11 - Reading for file: TRY_10 and angle: 10
2020-06-23 08:33:48.357 | INFO     | __main__:get_p_strain:11 - Reading for file: TRY_120 and angle: 120
2020-06-23 08:33:48.416 | INFO     | __main__:get_p_strain:11 - Reading for file: TRY_15 and angle: 15
2020-06-23 08:33:48.594 | INFO     | __main__:get_p_strain:11 - Reading for file: TRY_20 and angle: 20
2020-06-23 08:33:48.654 | INFO     | __main__:get_p_strain:11 - Reading for file: TRY_30 and angle: 30
2020-06-23 08:33:48.708 | INFO     | __main__:get_p_strain:11 - Reading for file: TRY_45 and angle: 45
2020-06-23 08:33:48.767 | INFO     | __main__:get_p_strain:11 - Reading for file: TRY_60 and angle: 60
2020-06-23 08:33:48.822 | INFO     | __main__:get_p_strain:11 - Rea

[('0', 0.0037064),
 ('10', 0.0031515),
 ('15', 0.0056231),
 ('20', 0.0025027),
 ('30', 0.005981),
 ('45', 0.0049572),
 ('60', 0.0017491),
 ('90', 0.0041059),
 ('120', 0.0055275)]

In [96]:
all_nodes_p_values

[[('0', 0.0018231),
  ('10', 0.0015416),
  ('15', 0.010005),
  ('20', 0.0014016),
  ('30', 0.0047554),
  ('45', 0.0097399),
  ('60', 0.0073354),
  ('90', 0.0093014),
  ('120', 0.0092186)],
 [('0', 0.0016167),
  ('10', 0.0014116),
  ('15', 0.0043106),
  ('20', 0.0012366),
  ('30', 0.0036154),
  ('45', 0.0044002),
  ('60', 0.0036055),
  ('90', 0.0040904),
  ('120', 0.0040486)],
 [('0', 0.00053447),
  ('10', 0.00048721),
  ('15', 0.002628),
  ('20', 0.00046256),
  ('30', 0.001529),
  ('45', 0.0023941),
  ('60', 0.0019233),
  ('90', 0.0023109),
  ('120', 0.0025296)],
 [('0', 0.0036156),
  ('10', 0.0031072),
  ('15', 0.0061698),
  ('20', 0.0025051),
  ('30', 0.0059702),
  ('45', 0.0041853),
  ('60', 0.0015207),
  ('90', 0.0045066),
  ('120', 0.0064266)],
 [('0', 0.0030323),
  ('10', 0.0025624),
  ('15', 0.0041954),
  ('20', 0.0020167),
  ('30', 0.0048393),
  ('45', 0.0037243),
  ('60', 0.0017166),
  ('90', 0.0033236),
  ('120', 0.0042197)],
 [('0', 0.0025168),
  ('10', 0.0019527),
  ('15', 